<a href="https://colab.research.google.com/github/sghub7/EMR-Statistics-Mon/blob/master/SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz


In [ ]:
!tar -xvf spark-3.1.2-bin-hadoop2.7.tgz

spark-3.1.2-bin-hadoop2.7/
spark-3.1.2-bin-hadoop2.7/R/
spark-3.1.2-bin-hadoop2.7/R/lib/
spark-3.1.2-bin-hadoop2.7/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdx
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdb
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/features.rds
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/pac

In [ ]:
!pip install -q findspark

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 50.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=bbfd726b4c5908949bd7f5cd12d1acbc225dfc7596ade4e9c6bcfd2b02e84870
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

Basic Statistics on Spark ML. 
Spark ML adds few extra data structures as part of ML Lib ->
Here we show Dense Vector , Sparse Vector , Matrices.

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
df = spark.createDataFrame(data, ["features"])

In [ ]:
df.show()

+--------------------+
|            features|
+--------------------+
|(4,[0,3],[1.0,-2.0])|
|   [4.0,5.0,0.0,3.0]|
|   [6.0,7.0,0.0,8.0]|
| (4,[0,3],[9.0,1.0])|
+--------------------+



In [ ]:
r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

Pearson correlation matrix:
DenseMatrix([[1.        , 0.05564149,        nan, 0.40047142],
             [0.05564149, 1.        ,        nan, 0.91359586],
             [       nan,        nan, 1.        ,        nan],
             [0.40047142, 0.91359586,        nan, 1.        ]])
Spearman correlation matrix:
DenseMatrix([[1.        , 0.10540926,        nan, 0.4       ],
             [0.10540926, 1.        ,        nan, 0.9486833 ],
             [       nan,        nan, 1.        ,        nan],
             [0.4       , 0.9486833 ,        nan, 1.        ]])


Matrix Multiplication Demo


In [ ]:
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import Matrices
sc= spark.sparkContext
dm1 = Matrices.dense(2, 3, [1, 2, 3, 4, 5, 6])
dm2 = Matrices.dense(2, 3, [7, 8, 9, 10, 11, 12])
dm3 = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])
dm4 = Matrices.dense(3, 2, [7, 8, 9, 10, 11, 12])
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 1, 2], [7, 11, 12])
blocks1 = sc.parallelize([((0, 0), dm1), ((0, 1), dm2)])
blocks2 = sc.parallelize([((0, 0), dm3), ((1, 0), dm4)])

mat1 = BlockMatrix(blocks1, 2, 3)
mat2 = BlockMatrix(blocks2, 3, 2)


In [ ]:
mat1.multiply(mat2).toLocalMatrix() ## Avoid to LocalMatrix () as it collect the distributed matrix on driver as a dense matrix

DenseMatrix(2, 2, [242.0, 272.0, 350.0, 398.0], 0)

Machine Learning on Spark ML

---


End to END ML using Spark ML 

---
AirBnb NYC 2019 Data Set and  Predict room prices




In [ ]:
from google.colab import files
uploaded = files.upload()


Saving AB_NYC_2019.csv to AB_NYC_2019.csv


Step 0 - Load and Clean Data

In [ ]:
## Doing some Basic EDA
airDF = spark.read.format("csv").option("header","true").option("multiLine", "true").option("inferSchema","true").load("AB_NYC_2019.csv")
airDF.repartition(3) ## Repartioning to distribute. Ideally when you read from HDFS files will be partioned already

airDF.cache()

DataFrame[id: int, name: string, host_id: string, host_name: string, neighbourhood_group: string, neighbourhood: string, latitude: string, longitude: string, room_type: string, price: string, minimum_nights: string, number_of_reviews: double, last_review: string, reviews_per_month: string, calculated_host_listings_count: string, availability_365: int]

In [ ]:
airDF.show(50,False)

+-----+--------------------------------------------------+-------+----------------+-------------------+-------------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|id   |name                                              |host_id|host_name       |neighbourhood_group|neighbourhood            |latitude|longitude|room_type      |price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+-----+--------------------------------------------------+-------+----------------+-------------------+-------------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539 |Clean & quiet apt home by the park                |2787   |John            |Brooklyn           |Kensington               |40.64749|-73.97237|Privat

In [ ]:
airDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: double (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [ ]:
## Describe the dataframe. 
## Explore Dataframe
##Describe function describe returns a DataFrame containing information such as number of non-null entries (count), mean, standard deviation, and minimum and maximum value for each numerical column
airDF.describe().show(1,False) ## Just displaying the count here

+-------+-----+-----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|summary|id   |name |host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+-------+-----+-----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|count  |48895|48879|48895  |48874    |48895              |48895        |48895   |48895    |48895    |48895|48895         |48895            |38843      |38843            |48895                         |48895           |
+-------+-----+-----+-------+---------+-------------------+-------------+--------+---------+---------+-----+------------

In [ ]:
## Neighbourhoods having the highest number of rentals
from pyspark.sql.functions import col
airDF.groupBy("neighbourhood").count().orderBy(col("count").desc()).show()


+------------------+-----+
|     neighbourhood|count|
+------------------+-----+
|      Williamsburg| 3920|
|Bedford-Stuyvesant| 3714|
|            Harlem| 2655|
|          Bushwick| 2465|
|   Upper West Side| 1971|
|    Hell's Kitchen| 1958|
|      East Village| 1853|
|   Upper East Side| 1798|
|     Crown Heights| 1563|
|           Midtown| 1545|
|       East Harlem| 1117|
|        Greenpoint| 1115|
|           Chelsea| 1113|
|   Lower East Side|  911|
|           Astoria|  900|
|Washington Heights|  899|
|      West Village|  767|
|Financial District|  744|
|          Flatbush|  621|
|      Clinton Hill|  572|
+------------------+-----+
only showing top 20 rows



In [ ]:
## Let's run some Statistics on price column
airDF.select("price").describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|             48895|
|   mean|152.72225387032114|
| stddev|240.16757937623433|
|    min|         -74.00828|
|    max|      Private room|
+-------+------------------+



In [ ]:
## Price of -73** and "private room" ??.. Let's explore
airDF.filter("price == 'Private room' or price == '-73.99986' ").show(10,False)

+--------+--------------------------------------+--------------+---------+-------------------+-------------+--------+---------+---------+------------+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|id      |name                                  |host_id       |host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price       |minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+--------+--------------------------------------+--------------+---------+-------------------+-------------+--------+---------+---------+------------+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|4681219 |"""Central Park"" View Prvt Room      |Mnh  "        |24208781 |William Hakan      |Manhattan    |Harlem  |40.80126 |-73.95777|Private room|70            |2.0              |68         |2019-05-19       |1

In [ ]:
## Let's drop these rows for now
airDF=airDF.filter("price != 'Private room' and  price != '-73.99986' ")


In [ ]:
airDF.select("price").describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|             48893|
|   mean|152.72225387032114|
| stddev|240.16757937623433|
|    min|         -74.00828|
|    max|         Manhattan|
+-------+------------------+



In [ ]:
airDF=airDF.filter("price not like '%-%' and price not like '%a%' ") ## A Crude way of data cleanin. There may be better ways

In [ ]:
airDF.select("price").describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|             48889|
|   mean|152.72689152979197|
| stddev|240.16784649612146|
|    min|                 0|
|    max|              9999|
+-------+------------------+



In [ ]:
#Let's convert  columns to Double type to perform some more analysis
from pyspark.sql.functions import expr
airDF = airDF.withColumn("priceDouble",expr("CAST (price as Double)")).drop("price").withColumnRenamed("priceDouble","price")
airDF = airDF.withColumn("minimum_nights_d",expr("CAST (minimum_nights as Double)")).drop("minimum_nights").withColumnRenamed("minimum_nights_d","minimum_nights")
airDF = airDF.withColumn("latitude_d",expr("CAST (latitude as Double)")).drop("latitude").withColumnRenamed("latitude_d","latitude")
airDF = airDF.withColumn("longitude_d",expr("CAST (longitude as Double)")).drop("longitude").withColumnRenamed("longitude_d","longitude")
airDF = airDF.withColumn("reviews_per_month_double",expr("CAST (reviews_per_month as Double)")).drop("reviews_per_month").withColumnRenamed("reviews_per_month_double","reviews_per_month")
airDF = airDF.withColumn("calculated_host_listings_count_d",expr("CAST (calculated_host_listings_count as Double)")).drop("calculated_host_listings_count").withColumnRenamed("calculated_host_listings_count_d","calculated_host_listings_count")

In [ ]:
airDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- number_of_reviews: double (nullable = true)
 |-- last_review: string (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- minimum_nights: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)



In [ ]:
## See some stats on the numeric columns
airDF.select("minimum_nights","number_of_reviews","reviews_per_month","latitude","longitude","calculated_host_listings_count","price").describe().show()

+-------+-----------------+------------------+------------------+-------------------+------------------+------------------------------+------------------+
|summary|   minimum_nights| number_of_reviews| reviews_per_month|           latitude|         longitude|calculated_host_listings_count|             price|
+-------+-----------------+------------------+------------------+-------------------+------------------+------------------------------+------------------+
|  count|            48889|             48889|             38837|              48889|             48889|                         48889|             48889|
|   mean|7.030436294462967|23.274438012640882|1.3731660529907315|  40.72894774120974|-73.95216762625476|             7.144633762195995|152.72689152979197|
| stddev|20.51173512492114| 44.55236615217496|1.6805070568202156|0.05452421651616377|0.0461582750776722|             32.95448440433164|240.16784649612146|
|    min|              1.0|               0.0|              0.01|     

In [ ]:
from pyspark.sql.functions import log
airDF = airDF.withColumn("log_price",log("price"))

In [ ]:
airDF.select("minimum_nights","number_of_reviews","reviews_per_month","latitude","longitude","calculated_host_listings_count","price","log_price").describe().show()
## This shows we have some nulls in price. Lets remove them

+-------+-----------------+------------------+------------------+-------------------+------------------+------------------------------+------------------+------------------+
|summary|   minimum_nights| number_of_reviews| reviews_per_month|           latitude|         longitude|calculated_host_listings_count|             price|         log_price|
+-------+-----------------+------------------+------------------+-------------------+------------------+------------------------------+------------------+------------------+
|  count|            48889|             48889|             38837|              48889|             48889|                         48889|             48889|             48878|
|   mean|7.030436294462967|23.274438012640882|1.3731660529907315|  40.72894774120974|-73.95216762625476|             7.144633762195995|152.72689152979197| 4.727052069177271|
| stddev|20.51173512492114| 44.55236615217496|1.6805070568202156|0.05452421651616377|0.0461582750776722|             32.9544844043

In [ ]:
## This shows we have some nulls in price. Lets remove them
airDF=airDF.filter(airDF.log_price. isNotNull())

In [ ]:
airDF.select("minimum_nights","number_of_reviews","reviews_per_month","latitude","longitude","calculated_host_listings_count","price","log_price").describe().show()

+-------+------------------+------------------+------------------+-------------------+-------------------+------------------------------+------------------+------------------+
|summary|    minimum_nights| number_of_reviews| reviews_per_month|           latitude|          longitude|calculated_host_listings_count|             price|         log_price|
+-------+------------------+------------------+------------------+-------------------+-------------------+------------------------------+------------------+------------------+
|  count|             48878|             48878|             38827|              48878|              48878|                         48878|             48878|             48878|
|   mean|7.0303613077458165|23.271962846270306| 1.373072346562986| 40.728951542207184| -73.95217358954885|            7.1452800851098655|152.76126273579115| 4.727052069177271|
| stddev|20.513409920704554|44.553114906053395|1.6804557707293224|0.05452591012206169|0.04616027941329833|            32

In [ ]:
## Lets impute reviews_per_month Cloumn
##  Imputation strategies -> Mean ( default) , Median , Mode ( Most common value in the column)

##Lets covert column to Numeric before imputing
airDF = airDF.withColumn("reviews_per_month_double",expr("CAST (reviews_per_month as Double)")).drop("reviews_per_month").withColumnRenamed("reviews_per_month_double","reviews_per_month")

airDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- number_of_reviews: double (nullable = true)
 |-- last_review: string (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- minimum_nights: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)
 |-- log_price: double (nullable = true)
 |-- reviews_per_month: double (nullable = true)



In [ ]:
# Define the imputer
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=["reviews_per_month"], outputCols=["reviews_per_month_imputed"])
model = imputer.fit(airDF)
airDF= model.transform(airDF)
airDF.show(10,False)


+----+------------------------------------------------+-------+-----------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+-----------------+-------------------------+
|id  |name                                            |host_id|host_name  |neighbourhood_group|neighbourhood     |room_type      |number_of_reviews|last_review|availability_365|price|minimum_nights|latitude|longitude|calculated_host_listings_count|log_price         |reviews_per_month|reviews_per_month_imputed|
+----+------------------------------------------------+-------+-----------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+-----------------+-------------------------+
|2539|Clean & quiet apt home by the park              |2787   |J

In [ ]:
## Drop reviews_per_month column and use the new imputed reviews_per_month column
airDF = airDF.drop ("reviews_per_month")

In [ ]:
airDF=airDF.withColumnRenamed("reviews_per_month_imputed","reviews_per_month")

In [ ]:
airDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- number_of_reviews: double (nullable = true)
 |-- last_review: string (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- minimum_nights: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- calculated_host_listings_count: double (nullable = true)
 |-- log_price: double (nullable = true)
 |-- reviews_per_month: double (nullable = true)



Step 1 - Featurization
Doc - https://spark.apache.org/docs/latest/ml-features.html

In [ ]:
# Encode Categorical Variables
from pyspark.ml.feature import StringIndexer

neighbourhood_i = StringIndexer(inputCol="neighbourhood", outputCol="neighbourhood_indexed", handleInvalid="skip")
neighbourhood_group_i = StringIndexer(inputCol="neighbourhood_group", outputCol="neighbourhood_group_indexed", handleInvalid="skip")
roomtype_i = StringIndexer(inputCol="room_type", outputCol="room_type_indexed", handleInvalid="skip")


In [ ]:
## See how the indexed column looks like
## This is just for checking and showcase purpose. We will asseble all stages finally in a pipeline object
index_neighbourhood_model = neighbourhood_i.fit(airDF)
airDF_dummy= index_neighbourhood_model.transform(airDF)
airDF_dummy.show(10,False)

+----+------------------------------------------------+-------+-----------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+------------------+---------------------+
|id  |name                                            |host_id|host_name  |neighbourhood_group|neighbourhood     |room_type      |number_of_reviews|last_review|availability_365|price|minimum_nights|latitude|longitude|calculated_host_listings_count|log_price         |reviews_per_month |neighbourhood_indexed|
+----+------------------------------------------------+-------+-----------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+------------------+---------------------+
|2539|Clean & quiet apt home by the park              |2787   |John      

In [ ]:
# Vectorize Encoded Columns
## This is just for checking and showcase purpose. We will asseble all stages finally in a pipeline object

from pyspark.ml.feature import OneHotEncoder
oneHotEnc_Neighbourhood= OneHotEncoder(
  inputCols=["neighbourhood_indexed"],
  outputCols=["oneHot_neighborhood"]
)
oneHotEnc_Neighbourhood_model = oneHotEnc_Neighbourhood.fit(airDF_dummy)
airDF_dummy_onehot = oneHotEnc_Neighbourhood_model.transform(airDF_dummy)
airDF_dummy_onehot.show()



+----+--------------------+-------+----------------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+------------------+---------------------+-------------------+
|  id|                name|host_id|       host_name|neighbourhood_group|     neighbourhood|      room_type|number_of_reviews|last_review|availability_365|price|minimum_nights|latitude|longitude|calculated_host_listings_count|         log_price| reviews_per_month|neighbourhood_indexed|oneHot_neighborhood|
+----+--------------------+-------+----------------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+------------------+---------------------+-------------------+
|2539|Clean & quiet apt...|   2787|            John|           Brooklyn|        Ke

In [ ]:
# Vectorize All Encoded Columns
oneHotEnc = OneHotEncoder(
  inputCols=["neighbourhood_indexed", "neighbourhood_group_indexed", "room_type_indexed"],
  outputCols=["oneHot_neighborhood", "oneHOt_neighbourhood_group", "oneHot_room_type"]
)

In [ ]:
## What are our feature columns
featuresColumns = [
 "reviews_per_month",
 "number_of_reviews",
 "availability_365",
 "minimum_nights",
 "latitude",
 "longitude",
 "oneHot_neighborhood",
 "oneHOt_neighbourhood_group",
 "oneHot_room_type"
]

In [ ]:
## Assemble our features in a Vector to pass to the ML 
## Vector assembler is a transformer that combines a given list of columns into a single vector column
## Spark ML expects the features in a single vector
from pyspark.ml.feature import VectorAssembler
vectorized_features = VectorAssembler(inputCols=featuresColumns, outputCol="Model_Features")

Step 3 - Create the ML Pipeline

In [ ]:
## We now put all our stages/steps peerformed earlier in a Spark ML Pipeline
## Spark will execute the pipeline in sequence


from pyspark.ml import Pipeline
features_pipe = Pipeline()
features_pipe.setStages([neighbourhood_i, neighbourhood_group_i, roomtype_i,oneHotEnc , vectorized_features])

Pipeline_ba7085a6a904

In [ ]:
## Let's see how the final data set looks
features_pipe.fit(airDF).transform(airDF).show()
## Model_Features will be the only input column for our model trainig

+----+--------------------+-------+----------------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+------------------+---------------------+---------------------------+-----------------+-------------------+--------------------------+----------------+--------------------+
|  id|                name|host_id|       host_name|neighbourhood_group|     neighbourhood|      room_type|number_of_reviews|last_review|availability_365|price|minimum_nights|latitude|longitude|calculated_host_listings_count|         log_price| reviews_per_month|neighbourhood_indexed|neighbourhood_group_indexed|room_type_indexed|oneHot_neighborhood|oneHOt_neighbourhood_group|oneHot_room_type|      Model_Features|
+----+--------------------+-------+----------------+-------------------+------------------+---------------+-----------------+-----------+----------------+-----+------

In [ ]:
## Train / test Split
## Split randon with a seed so you can reproduce the same train /test data as needed
seed = 51
(testDF, trainDF) = airDF.randomSplit((0.20, 0.80), seed=seed)

In [ ]:
testDF.count()

9610

In [ ]:
trainDF.count()

39277

In [ ]:
## Training
## NOTE : A pipeline can have many pipelines.
## Here we will now add a Linear Regression Estimator in the pipeline and create a new pipeline which includes our featurization pipeline
## Let;s use Price column as the label column and see teh metrics
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
lr = LinearRegression(labelCol="price", featuresCol="Model_Features")
air_pipeline_price = Pipeline(stages=[features_pipe, lr])



In [ ]:
## Lets fit on training Data
air_lr_model_price = air_pipeline_price.fit(trainDF)

In [ ]:
lr_price_Summary = air_lr_model_price.stages[-1].summary

In [ ]:
lr_price_Summary.r2

0.11768224332720201

In [ ]:
## Using log price as label
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
lr = LinearRegression(labelCol="log_price", featuresCol="Model_Features",predictionCol="predicted_log_price")
air_pipeline_lr = Pipeline(stages=[features_pipe, lr])

In [ ]:
air_lr_model = air_pipeline_lr.fit(trainDF)

In [ ]:
lr_Summary = air_lr_model.stages[-1].summary

In [ ]:
## A very better r2 score with log price 
lr_Summary.r2

0.5447723505100401

In [ ]:
## GB Tree Model
from pyspark.ml.regression import GBTRegressor
gb=GBTRegressor(labelCol="log_price", featuresCol="Model_Features",predictionCol="predicted_log_price")
air_pipeline = Pipeline(stages=[features_pipe, gb])
air_gb_model=air_pipeline.fit(trainDF)

In [ ]:
gb_Summary = air_gb_model.stages[-1]
gb_Summary.featureImportances

SparseVector(231, {0: 0.0452, 1: 0.0744, 2: 0.1372, 3: 0.0944, 4: 0.0708, 5: 0.0783, 6: 0.0122, 7: 0.0018, 8: 0.0011, 9: 0.0005, 10: 0.0047, 11: 0.0003, 12: 0.0056, 13: 0.001, 14: 0.0004, 15: 0.0053, 16: 0.0005, 17: 0.0003, 19: 0.0047, 21: 0.0015, 22: 0.0017, 25: 0.0044, 26: 0.0015, 28: 0.0007, 31: 0.001, 36: 0.0022, 37: 0.003, 38: 0.001, 39: 0.0028, 40: 0.0033, 41: 0.0019, 44: 0.0005, 45: 0.0014, 47: 0.0003, 49: 0.001, 53: 0.0001, 59: 0.0179, 60: 0.0005, 62: 0.0006, 64: 0.001, 65: 0.0015, 67: 0.0003, 71: 0.001, 79: 0.0018, 80: 0.0008, 81: 0.0005, 84: 0.0036, 87: 0.0007, 89: 0.0008, 105: 0.0005, 106: 0.0024, 120: 0.0009, 131: 0.0017, 134: 0.0005, 140: 0.0003, 144: 0.0017, 155: 0.0085, 181: 0.0002, 194: 0.0039, 206: 0.0022, 225: 0.027, 226: 0.0012, 227: 0.0016, 228: 0.0018, 229: 0.3328, 230: 0.0146})

Step 5 - Model Evaluation

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator= RegressionEvaluator()
print(evaluator.explainParams())

labelCol: label column name. (default: label)
metricName: metric name in evaluation - one of:
                       rmse - root mean squared error (default)
                       mse - mean squared error
                       r2 - r^2 metric
                       mae - mean absolute error
                       var - explained variance. (default: rmse)
predictionCol: prediction column name. (default: prediction)
throughOrigin: whether the regression is through the origin. (default: False)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)


In [ ]:
evaluator.setLabelCol("log_price")
evaluator.setPredictionCol("predicted_log_price")


RegressionEvaluator_656ac2e6b64c

In [ ]:
evaluator.evaluate(air_lr_model.transform(testDF))

0.4687019953405141

In [ ]:
##Slightly Better performance with GBT
evaluator.evaluate(air_gb_model.transform(testDF))

0.4493783420789403

Step 6 -  Model persistence and Inference

In [ ]:
predctionsDF=air_gb_model.transform(testDF)

In [ ]:
predctionsDF.show()

+-----+--------------------+-------+---------------+-------------------+--------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+------------------+---------------------+---------------------------+-----------------+-------------------+--------------------------+----------------+--------------------+-------------------+
|   id|                name|host_id|      host_name|neighbourhood_group|       neighbourhood|      room_type|number_of_reviews|last_review|availability_365|price|minimum_nights|latitude|longitude|calculated_host_listings_count|         log_price| reviews_per_month|neighbourhood_indexed|neighbourhood_group_indexed|room_type_indexed|oneHot_neighborhood|oneHOt_neighbourhood_group|oneHot_room_type|      Model_Features|predicted_log_price|
+-----+--------------------+-------+---------------+-------------------+--------------------+---------------+-------------

In [ ]:
## Saving the model for  inference later
air_gb_model.save("/content/models")

In [ ]:
!ls

AB_NYC_2019.csv  models       spark-3.1.2-bin-hadoop2.7
listings.csv	 sample_data  spark-3.1.2-bin-hadoop2.7.tgz


In [ ]:
!ls models

metadata  stages


In [ ]:
!ls models/stages

0_PipelineModel_f4ed09c47006  1_GBTRegressor_cf21c5dc6a2c


In [ ]:
## Loading the model back from storage
from pyspark.ml.pipeline import PipelineModel
loaded_gb_model = PipelineModel.load("/content/models")

In [ ]:
predctionsDF=loaded_gb_model.transform(testDF)

In [ ]:
predctionsDF.show()

+-----+--------------------+-------+---------------+-------------------+--------------------+---------------+-----------------+-----------+----------------+-----+--------------+--------+---------+------------------------------+------------------+------------------+---------------------+---------------------------+-----------------+-------------------+--------------------------+----------------+--------------------+-------------------+
|   id|                name|host_id|      host_name|neighbourhood_group|       neighbourhood|      room_type|number_of_reviews|last_review|availability_365|price|minimum_nights|latitude|longitude|calculated_host_listings_count|         log_price| reviews_per_month|neighbourhood_indexed|neighbourhood_group_indexed|room_type_indexed|oneHot_neighborhood|oneHOt_neighbourhood_group|oneHot_room_type|      Model_Features|predicted_log_price|
+-----+--------------------+-------+---------------+-------------------+--------------------+---------------+-------------

Hyperparameter/ Model Tuning


In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
paramGrid = (ParamGridBuilder()
  .addGrid(lr.elasticNetParam, [0., 1.])
  .addGrid(lr.maxIter, [1, 10, 100])
  .addGrid(lr.fitIntercept, [True, False])
  .build()
)

In [ ]:
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(
  estimator = air_pipeline_lr,         
  estimatorParamMaps = paramGrid,   
  evaluator=evaluator,             
  numFolds = 3
)

In [ ]:
cvModel_lr = cv.fit(trainDF)

In [ ]:
evaluator.evaluate(cvModel_lr.transform(testDF))

0.4687019953405141